In [ ]:
#pip install contextily

In [ ]:
#pip install geopandas

In [ ]:
#pip install PyQt5

In [ ]:
import math
import networkx as nx
import matplotlib.pyplot as plt
import contextily as cx
import geopandas as gpd

def parse_co_file(filename, graph):
    pos = dict()
    with open(filename) as file:
        for line in file:
            if line[0] == 'v': # v <node> <x> <y>
                line = line.split()
                node = int(line[1])
                x = float(line[2]) / 1e6 # wgs84 decimal
                y = float(line[3]) / 1e6 # wgs84 decimal
                pos[node] = (x, y)
            elif line[0] == 'c': # c comment
                continue
            elif line[0] == 'p': # p aux sp co <nodes>
                continue
    print("Parsed", len(pos), "node positions from", filename)
    for node in pos:
        graph.add_node(node, pos = pos[node])
    return pos

def parse_gr_file(filename, graph):
    edges = 0
    with open(filename) as file:
        for line in file:
            if line[0] == 'c': # c comment
                continue
            elif line[0] == 'p': # p sp <nodes> <edges>
                continue
            elif line[0] == 'a': # a <src> <dst> <weight>
                line = line.split()
                src = int(line[1])
                dst = int(line[2])
                weight = int(line[3])
                graph.add_edge(src, dst, weight = weight)
                edges += 1
    print("Parsed", edges, "edges from", filename)
    return graph

def calc_heuristic(graph, positions, destination):
    for node in graph.nodes():
        graph.nodes[node]['heuristic'] = math.sqrt((positions[node][0] - positions[destination][0])**2 + (positions[node][1] - positions[destination][1])**2)

def plot_map(graph, west, south, east, north,  path = None, showEdges = False):
    section = {node for node in graph.nodes() if graph.nodes[node]['pos'][0] > west and graph.nodes[node]['pos'][0] < east and graph.nodes[node]['pos'][1] > south and graph.nodes[node]['pos'][1] < north}
    posx = [graph.nodes[node]['pos'][0] for node in section]
    posy = [graph.nodes[node]['pos'][1] for node in section]
    fig, ax = plt.subplots(1, 1, figsize=(30, 30))
    pos_gdf = gpd.GeoDataFrame(section, geometry=gpd.points_from_xy(posx, posy), crs='EPSG:4326')
    ax = pos_gdf.plot(ax=ax, color='blue', markersize=100, alpha=0.5)
    cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.OpenStreetMap.Mapnik)
    if showEdges:
        jfk_edges = [(src, dst) for src, dst in graph.edges() if src in section and dst in section]
        nx.draw_networkx_edges(graph, pos = nx.get_node_attributes(graph, 'pos'), edgelist = jfk_edges, node_size = 10, ax = ax, edge_color = 'blue', alpha = 0.5)
    nx.draw_networkx_labels(graph, pos = nx.get_node_attributes(graph, 'pos'), labels = {node: node for node in section}, font_size = 4, ax = ax)
    if path is not None:
        plt.savefig('.\\data\\jfk.png')
    return fig

def plot_path(graph, path):
    posx = [graph.nodes[node]['pos'][0] for node in path]
    posy = [graph.nodes[node]['pos'][1] for node in path]
    fig, ax = plt.subplots(1, 1, figsize=(30, 30))
    pos_gdf = gpd.GeoDataFrame(path, geometry=gpd.points_from_xy(posx, posy), crs='EPSG:4326')
    ax = pos_gdf.plot(ax=ax)
    cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.OpenStreetMap.Mapnik)   
    return fig


In [ ]:
prefix = 'NY'
graph = nx.DiGraph()         
parse_co_file('.\\data\\USA-road-d.' + prefix + '.co', graph)
parse_gr_file('.\\data\\USA-road-t.' + prefix + '.gr', graph)

In [ ]:
west, south, east, north = (-73.81, 40.640, -73.77, 40.66)
fig = plot_map(graph,-73.81, 40.640, -73.77, 40.66, '.\\data\\jfk.png')

In [ ]:
west, south, east, north = (-73.89, 40.768, -73.856, 40.785)
fig = plot_map(graph, -73.89, 40.768, -73.856, 40.785, '.\\data\\laguardia.png')

In [ ]:
west, south, east, north = (-74.19, 40.686, -74.17, 40.697)
fig = plot_map(graph, -73.89, 40.768, -73.856, 40.785, '.\\data\\newark.png')

In [ ]:
jfk_node = 212410
laguardia_node = 198373
newark_node = 49611

In [ ]:
import heapq 
#use heapq.heappop(listname) to retrieve an element
#use heapq.heappush(listname, (key, value)) to store an element in the heap
#don't perform other list manipulations as those could destroy the heap property  

def dijkstra(graph, src, dst):
    return (distance, pred)

def construct_path(pred, start, end):
    return path

In [ ]:
(distance, pred) = dijkstra(graph, jfk_node, laguardia_node)
path1 = construct_path(pred, jfk_node, laguardia_node)
print("JFK to LaGuardia:", distance, path1)
fig = plot_path(graph, path1)

In [ ]:
(distance, pred) = dijkstra(graph, laguardia_node, newark_node)
path2 = construct_path(pred, laguardia_node, newark_node)
print("LaGuardia to NewArk:", distance, path2)
fig = plot_path(graph, path2)

In [ ]:
(distance, pred) = dijkstra(graph, newark_node, jfk_node)
path3 = construct_path(pred, newark_node, jfk_node)
print("NewArk to JFK:", distance, path3)
fig = plot_path(graph, path3)

In [ ]:
roundtrip = path1+path2+path3
fig = plot_path(graph, roundtrip)